In [17]:
import pandas as pd
import joblib
import json
import warnings

# Tắt cảnh báo UserWarning
warnings.filterwarnings("ignore", category=UserWarning)

# Đường dẫn tới tệp JSON
ATTACK_FEATURES_FILE = 'GA_output_ET.json'

# Hàm tải attack_features từ tệp JSON
def load_attack_features(json_file_path):
    with open(json_file_path, 'r') as file:
        attack_features = json.load(file)
    return attack_features

# Tải attack_features từ JSON
attack_features = load_attack_features(ATTACK_FEATURES_FILE)

model_paths =  {
    'SYN': './models/DT_SYN_1_model.pkl',
    'HTTP': './models/DT_HTTP_1_model.pkl',
    'ACK': './models/DT_ACK_1_model.pkl',
    'UDP': './models/DT_UDP_1_model.pkl',
    'ARP': './models/DT_ARP_1_model.pkl',
    'SP': './models/DT_SP_1_model.pkl',
    'BF': './models/DT_BF_1_model.pkl',
    'OS': './models/DT_OS_1_model.pkl',
    'SCHD': './models/DT_SCHD_1_model.pkl',
    'MHDis': './models/DT_MHDis_1_model.pkl'
}

# Hàm dự đoán kiểu tấn công
def predict_attack(input_data):
    results = []
    for attack, model_path in model_paths.items():
        # Tải mô hình và đặc trưng tương ứng
        model = joblib.load(model_path)
        features = attack_features[attack]
        
        # Lọc dữ liệu đầu vào theo đặc trưng
        filtered_data = input_data[features[:-1]]  # Bỏ cột 'Label'
        
        # Dự đoán
        predictions = model.predict(filtered_data)
        probabilities = model.predict_proba(filtered_data)[:, 1]
        
        # Lưu kết quả
        for pred, prob in zip(predictions, probabilities):
            result = {"Attack_Type": attack, "Prediction": int(pred), "Probability": float(prob)}

            if result not in results:
                results.append(result)
    
    return results

# Tổng hợp kết quả
def summarize_results(results):
    import pandas as pd
    results_df = pd.DataFrame(results)
    attack_predictions = results_df[results_df["Prediction"] == 1]
    
    if not attack_predictions.empty:
        # Trả về toàn bộ các kiểu tấn công được phát hiện
        attack_predictions = attack_predictions.sort_values(by="Probability", ascending=False)
        return attack_predictions.to_dict(orient="records")
    else:
        return []

# Chạy chương trình
if __name__ == "__main__":
    # Tải dữ liệu đầu vào để dự đoán
    test_data = pd.read_csv("./new_attack_data.csv")
    
    # Dự đoán
    results = predict_attack(test_data)
    
    # Tổng hợp kết quả
    final_results = summarize_results(results)
    
    if final_results:
        print("Detected Attacks:")
        for attack in final_results:
            print(f" - Attack Type: {attack['Attack_Type']}, Probability: {attack['Probability']}")
    else:
        print("No attacks detected.")

Detected Attacks:
 - Attack Type: SYN, Probability: 1.0
 - Attack Type: SP, Probability: 1.0
 - Attack Type: SCHD, Probability: 1.0
 - Attack Type: OS, Probability: 0.9990319457889641
 - Attack Type: ACK, Probability: 0.6918844566712518
